## Walking Through the Basic Workflow of Sci-Kit Learn

* start-finish Workflow
* 1: Getting the data ready
* 2: Choose the right model settings for the data
* 3: Fit the Model
* 4: Evaluating the Model
* 5: Improve the Model

In [2]:
# 1: Getting the data ready
import pandas as pd
import numpy as np
df= pd.read_csv('heart-disease.csv')

In [5]:
X = df.drop('target', axis=1)
y = df['target']
print(X.head(),y.head())

   age  sex  cp  trestbps  chol  fbs  restecg  thalach  exang  oldpeak  slope  \
0   63    1   3       145   233    1        0      150      0      2.3      0   
1   37    1   2       130   250    0        1      187      0      3.5      0   
2   41    0   1       130   204    0        0      172      0      1.4      2   
3   56    1   1       120   236    0        1      178      0      0.8      2   
4   57    0   0       120   354    0        1      163      1      0.6      2   

   ca  thal  
0   0     1  
1   0     2  
2   0     2  
3   0     2  
4   0     2   0    1
1    1
2    1
3    1
4    1
Name: target, dtype: int64


In [7]:
#2 Choose the right model settings for the data
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier()
rfc.get_params()
# We will stick with the boilerplate parameters for now.

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': None,
 'max_features': 'auto',
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_impurity_split': None,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 100,
 'n_jobs': None,
 'oob_score': False,
 'random_state': None,
 'verbose': 0,
 'warm_start': False}

In [10]:
#We need to split our data into training and test sets
from sklearn.model_selection import train_test_split
xtrain, xtest, ytrain, ytest = train_test_split(X,y, test_size=.2)


In [ ]:
# 3: Fitting our data to the model
rfc.fit(xtrain, ytrain);
'''Notice that the training sets were used as opposed to the test sets. We need to keep them separate so the 
model doesn't cheat'''

In [14]:
import numpy as np
rfc.predict(np.array([0,4,5,2]).reshape(-1,1))

ValueError: X has 1 features, but DecisionTreeClassifier is expecting 13 features as input.

In [15]:
# Predictions should look like the training set rows with 13 'features':
xtrain.head(5)

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal
210,57,1,2,128,229,0,0,150,0,0.4,1,1,3
200,44,1,0,110,197,0,0,177,0,0.0,2,1,2
51,66,1,0,120,302,0,0,151,0,0.4,1,0,2
150,66,1,0,160,228,0,0,138,0,2.3,2,0,1
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1


In [17]:
#The model has the training set down pat, lets find out how it fares on the test data
rfc.score(xtrain,ytrain)

1.0

In [19]:
# 4: Evaluating the Model

y_preds = rfc.predict(xtest)
# the model is fed values to extrapolate from. Let's see how it did
rfc.score(xtest, ytest)

0.7868852459016393

In [23]:
# Let's find out a bit more about how it's doing
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
print(classification_report(y_preds,ytest))

              precision    recall  f1-score   support

           0       0.79      0.77      0.78        30
           1       0.78      0.81      0.79        31

    accuracy                           0.79        61
   macro avg       0.79      0.79      0.79        61
weighted avg       0.79      0.79      0.79        61



In [24]:
print(confusion_matrix(y_preds,ytest))

[[23  7]
 [ 6 25]]


In [26]:
print(accuracy_score(y_preds,ytest))

0.7868852459016393


In [29]:
# 5: Improve the Model

# we will change the amount of n estimators
for i in [10,100,1000]:
    print(f'model with {i} estimators')
    rfc = RandomForestClassifier(n_estimators=i)
    rfc.fit(xtrain,ytrain)
    y_preds= rfc.predict(xtest)
    print(accuracy_score(y_preds,ytest))
#10 estimators is marginally better

model with 10 estimators
0.8360655737704918
model with 100 estimators
0.7704918032786885
model with 1000 estimators
0.7868852459016393


In [32]:
# 6: save_model and load it
from joblib import dump, load
dump(rfc, 'rf_heart_disease_model.joblib')

['rf_heart_disease_model.joblib']

In [37]:
ld_model =load('rf_heart_disease_model.joblib')
ld_model.score(xtest,ytest)

0.7868852459016393

## 1. Getting the Data Ready

In [40]:
#1.1: making sure the data is numerical
df = pd.read_csv('car-sales-extended.csv')
df.head()

,Make,Colour,Odometer (KM),Doors,Price
0,Honda,White,35431,4,15323
1,BMW,Blue,192714,5,19943
2,Honda,White,84714,4,28343
3,Toyota,White,154365,4,13434
4,Nissan,Blue,181577,3,14043


In [41]:
df.dtypes

Make             object
Colour           object
Odometer (KM)     int64
Doors             int64
Price             int64
dtype: object

In [48]:
#splitting the data into X,y
X = df.drop('Price', axis=1)
y= df['Price']
# Test-Train split
xtrain, xtest, ytrain, ytest = train_test_split(X,y,test_size=0.2)

In [51]:
# set up our model
from sklearn.ensemble import RandomForestRegressor
rfr = RandomForestRegressor()
rfr.fit(xtrain,ytrain)
rfr.score(xtest,ytest)

ValueError: could not convert string to float: 'Toyota'

In [56]:
#Treating the categorical data -> Convert to numerical
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

categorical_features = ['Make', 'Colour', 'Doors']
''' We encode categorical data from doors because there is not a linear relationship.
There is an intangible difference between 3, 4, and 5 doors on a car'''
one_hot = OneHotEncoder()
transformer = ColumnTransformer([('one_hot',
                                 one_hot,
                                 categorical_features)],
                               remainder='passthrough')
transformed_X = transformer.fit_transform(X)
X_T = pd.DataFrame(data=transformed_X)
X_T

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,35431.0
1,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,192714.0
2,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,84714.0
3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,154365.0
4,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,181577.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,35820.0
996,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,155144.0
997,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,66604.0
998,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,215883.0


In [61]:
# Alternative to One Hot encoder:
dummies = pd.get_dummies(df[['Make', 'Colour', 'Doors']])
alt_X = pd.concat([dummies,df['Odometer (KM)']], axis=1)
alt_X.head()

,Doors,Make_BMW,Make_Honda,Make_Nissan,Make_Toyota,Colour_Black,Colour_Blue,Colour_Green,Colour_Red,Colour_White,Odometer (KM)
0,4,0,1,0,0,0,0,0,0,1,35431
1,5,1,0,0,0,0,1,0,0,0,192714
2,4,0,1,0,0,0,0,0,0,1,84714
3,4,0,0,0,1,0,0,0,0,1,154365
4,3,0,0,1,0,0,1,0,0,0,181577


In [65]:
# Refitting the model
xtrain,xtest,ytrain,ytest = train_test_split(X_T, y, test_size=0.2)
rfr.fit(xtrain,ytrain)
rfr.score(xtest,ytest)


0.26355342154597783

In [81]:
#import data with missing values (NaN)
missing_data = pd.read_csv('car-sales-extended-missing-data.csv')
missing_data.head()

,Make,Colour,Odometer (KM),Doors,Price
0,Honda,White,35431.0,4.0,15323.0
1,BMW,Blue,192714.0,5.0,19943.0
2,Honda,White,84714.0,4.0,28343.0
3,Toyota,White,154365.0,4.0,13434.0
4,Nissan,Blue,181577.0,3.0,14043.0


In [68]:
missing_data.isna().sum()

Make             49
Colour           50
Odometer (KM)    50
Doors            50
Price            50
dtype: int64

In [70]:
# Load new x,y
X = missing_data.drop('Price', axis=1)
y = missing_data['Price']

KeyError: "['Price'] not found in axis"

In [72]:
#retest the regression with the new data
#Treating the categorical data -> Convert to numerical

categorical_features = ['Make', 'Colour', 'Doors']
''' We encode categorical data from doors because there is not a linear relationship.
There is an intangible difference between 3, 4, and 5 doors on a car'''
one_hot = OneHotEncoder()
transformer = ColumnTransformer([('one_hot',
                                 one_hot,
                                 categorical_features)],
                               remainder='passthrough')
transformed_X = transformer.fit_transform(X)
X_T = pd.DataFrame(data=transformed_X)
X_T

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,35431.0
1,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,192714.0
2,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,84714.0
3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,154365.0
4,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,181577.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,35820.0
996,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,155144.0
997,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,66604.0
998,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,215883.0


# Though onehotencoder can handle nan, we should fill or drop it as a rule
# Pandas method:

In [82]:
# Reload the dataset
missing_data = pd.read_csv('car-sales-extended-missing-data.csv')

missing_data['Colour'].fillna('missing', inplace=True)
missing_data['Make'].fillna('missing', inplace=True)
missing_data['Odometer (KM)'].fillna(missing_data['Odometer (KM)'].mean(), inplace=True)
missing_data['Doors'].fillna(4, inplace=True)
missing_data.isna().sum()

Make              0
Colour            0
Odometer (KM)     0
Doors             0
Price            50
dtype: int64

In [83]:
missing_data.dropna(inplace=True)
len(missing_data.index)

950

In [85]:
missing_data.head()

,Make,Colour,Odometer (KM),Doors,Price
0,Honda,White,35431.0,4.0,15323.0
1,BMW,Blue,192714.0,5.0,19943.0
2,Honda,White,84714.0,4.0,28343.0
3,Toyota,White,154365.0,4.0,13434.0
4,Nissan,Blue,181577.0,3.0,14043.0


In [91]:
X = missing_data.drop('Price', axis=1)
y = missing_data['Price']
print(X.head(),y.head())

     Make Colour  Odometer (KM)  Doors
0   Honda  White        35431.0    4.0
1     BMW   Blue       192714.0    5.0
2   Honda  White        84714.0    4.0
3  Toyota  White       154365.0    4.0
4  Nissan   Blue       181577.0    3.0 0    15323.0
1    19943.0
2    28343.0
3    13434.0
4    14043.0
Name: Price, dtype: float64


In [92]:

categorical_features = ['Make', 'Colour', 'Doors']
''' We encode categorical data from doors because there is not a linear relationship.
There is an intangible difference between 3, 4, and 5 doors on a car'''
one_hot = OneHotEncoder()
transformer2 = ColumnTransformer([('one_hot',
                                 one_hot,
                                 categorical_features)],
                               remainder='passthrough')
transformed_X2 = transformer2.fit_transform(missing_data)
X_T2 = pd.DataFrame(data=transformed_X2)
X_T2

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,35431.0,15323.0
1,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,192714.0,19943.0
2,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,84714.0,28343.0
3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,154365.0,13434.0
4,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,181577.0,14043.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
945,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,35820.0,32042.0
946,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,155144.0,5716.0
947,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,66604.0,31570.0
948,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,215883.0,4001.0
